In [ ]:
import numpy as np
import pandas as pd
from micom import Community

In [ ]:
# Pseudo-dynamics settings
T_MAX_HOURS = 24.0   # start with shorter run for testing
DT_HOURS = 1.0       # 1-hour steps to keep it simple
TRADEOFF = 0.5       # MICOM cooperative tradeoff fraction

In [ ]:
AGORA_MODELS = {
    "A_muciniphila": "microbia-modeling/Akkermansia_muciniphila_ATCC_BAA_835.xml",
    "F_prausnitzii": "microbia-modeling/Faecalibacterium_prausnitzii_A2_165.xml",
    "A_caccae": "microbia-modeling/Anaerostipes_caccae_DSM_14662.xml",
    "E_hallii": "microbia-modeling/Eubacterium_hallii_DSM_3353.xml",
}

# set initial abundances (fractions summing to 1)
INITIAL_ABUNDANCES = {
    "A_muciniphila": 0.3,
    "F_prausnitzii": 0.25,
    "A_caccae":      0.25,
    "E_hallii":      0.2,
}

species_ids = list(AGORA_MODELS.keys())
species_ids

In [ ]:
#build MICOM community

tax = pd.DataFrame({
    "id": list(AGORA_MODELS.keys()),
    "file": list(AGORA_MODELS.values()),
    "abundance": [INITIAL_ABUNDANCES[s] for s in AGORA_MODELS.keys()],
})
tax

In [ ]:
# build the community
comm = Community(tax, id="Akk_crossfeeders")
comm

In [ ]:
# see the change of growth rate when one species is knocked out
comm.knockout_taxa(diag=False)

In [ ]:
# from the mechanism:akk produce acetate, propionate, and 1,2-propanediol.
# Other three crossfeeders produce acetate, butyrate and propionate.
EX_ACETATE     = "EX_ac_m"          # acetate
EX_BUTYRATE    = "EX_but_m"         # butyrate
EX_PROPIONATE  = "EX_ppa_m"         # propionate
EX_PROPANEDIOL = "EX_12ppd_S_m"     # 1,2-propanediol

In [ ]:
# see the optimal growth rates and fluxes
sol = comm.cooperative_tradeoff(fluxes=True, pfba=True, fraction=0.6)
sol

#### ----------------------------------------------------
#### Adjust medium: mucin sugars + Vitamin B12 supplement
#### ----------------------------------------------------
##### Some of these carbohydrates (fucose, galactose, N‐AcGam, N‐AcGal) are the building blocks of mucin, a key component of the intestinal mucus that protects the gastrointestinal epithelium. The mucus layer serves as a functional barrier, protecting the epithelium from mechanical stresses and direct contact with pathogens. Additionally, it lubricates the passage of food and ensures the diffusion of small molecules (nutrients, ions, water and gases) (Paone & Cani, 2020)

##### F. prausnitzii preferentially consumes monosaccharides (glucose, rhamnose, fucose, galactose), disaccharides (cellobiose, lactose), amino sugars (N‐acetylglucosamine (N‐AcGam), N‐acetylgalactosamine (N‐AcGal)) and certain polysaccharides (pectin, starch) (Hu et al., 2022; Lopez‐Siles et al., 2012).

In [ ]:
# -----------------------------
# Setup: IDs & baseline medium
# -----------------------------

# B12/cobalt in medium
EX_CBL1_M      = "EX_cbl1_m"
EX_CBL2_M      = "EX_cbl2_m"
EX_COBALT2_M   = "EX_cobalt2_m"
EX_ADOCBL_M    = "EX_adocbl_m"

# Mucin sugar in medium
MUCIN_PROXIES = [
    "EX_acgal_m",   # N-acetylgalatosamine
    "EX_acgam_m",   # N-acetylglucosamine
    "EX_fuc_L_m",   # L-fucose
    "EX_gal_m",     # galatose
    "EX_man_m",     # mannose
]


# Exchange reaction IDs for metabolites 
EX_ACETATE     = "EX_ac_m"          # acetate
EX_BUTYRATE    = "EX_but_m"         # butyrate
EX_PROPIONATE  = "EX_ppa_m"         # propionate
EX_PROPANEDIOL = "EX_12ppd_S_m"     # 1,2-propanediol


# Keywords for detecting B12-related exchanges in the medium
B12_KEYWORDS = ["b12", "cobal", "cbl", "vitb", "cobalamin", "cobalt"]

# the original MICOM medium
baseline_medium = comm.medium.copy()

# -----------------------------
# 1. Medium builder
# -----------------------------

def build_medium(low_mucin: bool = True,
                 low_b12: bool = True,
                 mucin_level: float = 0.001,
                 b12_level: float = 0.001):
    """
    Build a medium starting from the baseline, with options to:
      - low mucin sugar (acgam, fucose)
      - low B12/cobalt-related exchanges
    """
    med = baseline_medium.copy()

    # 1) Mucin-like substrates
    if low_mucin:
        for ex in MUCIN_PROXIES:
            if ex in med:
                med[ex] = mucin_level
                print(f"[mucin] {ex} = {mucin_level}")
            else:
                print(f"[mucin] {ex} not found in medium")

    # 2) low B12 / cobalt 
    if low_b12:
        b12_exchanges = [
            k for k in med.keys()
            if any(kw in k.lower() for kw in B12_KEYWORDS)
        ]
        print("[B12] Detected exchanges:", b12_exchanges)
        for ex in b12_exchanges:
            med[ex] = b12_level
            print(f"[B12] {ex} = {b12_level}")

    return med


# -----------------------------
# 2. Extract metabolite flux DataFrame for a single solution
# -----------------------------

def extract_metabolite_fluxes(fluxes: pd.DataFrame) -> pd.DataFrame:
    """
    From sol.fluxes (rows=compartments, cols=reactions),
    extract only the metabolites of interest (SCFAs, 1,2-PPD, B12/cobalt).

    Returns:
        df_flux: DataFrame [compartment x reaction]
    """
    ids = [
        EX_ACETATE,
        EX_BUTYRATE,
        EX_PROPIONATE,
        EX_PROPANEDIOL
    ]

    valid = [rid for rid in ids if rid in fluxes.columns]
    print("Valid metabolite exchange reactions in this solution:")
    print(valid)

    if not valid:
        print("⚠ No valid metabolite exchange reactions found.")
        return pd.DataFrame()

    df_flux = fluxes[valid].copy()
    df_flux.index.name = "compartment"
    df_flux.columns.name = "reaction"
    return df_flux


def add_net_community_row(df_flux: pd.DataFrame) -> pd.DataFrame:
    """
    Add a 'net_community_flux' row = sum of all species (excluding 'medium').
    """
    if df_flux.empty:
        return df_flux

    net_flux = df_flux.drop(index="medium", errors="ignore").sum(axis=0)
    net_flux.name = "net_community_flux"
    df_full = pd.concat([df_flux, net_flux.to_frame().T])
    return df_full


# -----------------------------
# 3. Run a condition (medium -> MICOM -> fluxes)
# -----------------------------

def run_condition(label: str,
                  low_mucin: bool = False,
                  low_b12: bool = False,
                  fraction: float = 0.3):
    """
    Build a medium with specified options, run cooperative_tradeoff, and
    return:
        - community growth
        - full metabolite flux dataframe (per species + net community)
    """
    print(f"\n=== Condition: {label} ===")
    med = build_medium(low_mucin=low_mucin, low_b12=low_b12)
    comm.medium = med

    # Solve community growth
    sol = comm.cooperative_tradeoff(fraction=fraction, pfba=True, fluxes=True)
    if sol is None:
        print("❌ No feasible solution.")
        return {
            "label": label,
            "growth": None,
            "fluxes": None,
            "df_flux": None,
        }

    print("Community growth rate:", sol.growth_rate)

    fluxes = sol.fluxes
    if fluxes is None:
        print("⚠ No fluxes returned by this MICOM version.")
        return {
            "label": label,
            "growth": sol.growth_rate,
            "fluxes": None,
            "df_flux": None,
        }

    # Extract only metabolites of interest and add net community row
    df_flux = extract_metabolite_fluxes(fluxes)
    df_full = add_net_community_row(df_flux)

    print("\nFlux dataframe (including net_community_flux):")
    print(df_full)

    return {
        "label": label,
        "growth": sol.growth_rate,
        "fluxes": fluxes,
        "df_flux": df_full,
    }

In [ ]:
# -----------------------------
# 4. Run multiple scenarios
# -----------------------------

# Baseline: original medium
cond_baseline = run_condition("Baseline", low_mucin=False, low_b12=False)

# Low B12 only
cond_b12 = run_condition("Low B12", low_mucin=False, low_b12=True)

# low mucin only
cond_mucin = run_condition("Low mucin", low_mucin=True, low_b12=False)

# Low mucin + low B12
cond_both = run_condition("Low mucin + Low B12", low_mucin=True, low_b12=True)

# -----------------------------
# 5. Optional: compare net flux across conditions
# -----------------------------

def get_net_series(cond):
    if cond["df_flux"] is None:
        return None
    df = cond["df_flux"]
    if "net_community_flux" not in df.index:
        return None
    s = df.loc["net_community_flux"].copy()
    s.name = cond["label"]
    return s

net_series = [get_net_series(c) for c in [cond_baseline, cond_mucin, cond_b12, cond_both]]
net_series = [s for s in net_series if s is not None]

if net_series:
    df_net = pd.DataFrame(net_series)
    print("\n=== Net community flux per condition ===")
    print(df_net)
else:
    print("\nNo net flux data available to compare.")
